In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date
from openpyxl import load_workbook
import re
import os
import gc

In [2]:
# Function to extract emails from a list of text strings
# def is_valid_email(sheet_name):
#     # Regular expression to match an email
#     email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
#     if sheet_name is None:
#             return False
#     match = re.search(email_pattern, sheet_name)
#     if match:
#         return True
#     return False


def is_valid_email(sheet_name):
    # Regular expression to match an email (allowing no domain extension)
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+'
    
    if sheet_name is None:
        return False
    
    match = re.search(email_pattern, sheet_name)
    if match:
        return True
    return False


In [3]:
# def get_processed_dataset(file_paths):
#     sheets_data = {}
#     for file_path in file_paths:
#         path_dataset=filter_and_merge_dataset_sheets(file_path)
#         sheets_data[file_path]=path_dataset

#     return pd.concat(sheets_data.values(), ignore_index=True)


def get_processed_dataset(folder_path):
    file_paths=[]
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_paths.append(file_name)
    print(f"Folder Path : {folder_path}")
    print(f"File Paths : {file_paths}")

    sheets_data = {}
    if len(file_paths)>0:
        for file_path in file_paths:
            path_dataset=filter_and_merge_dataset_sheets(folder_path+file_path)
            sheets_data[folder_path+file_path]=path_dataset

    return pd.concat(sheets_data.values(), ignore_index=True)
    

In [4]:
def filter_and_merge_dataset_sheets(file_path):
    wb = load_workbook(file_path, data_only=True)
    # Dictionary to store DataFrames for each sheet
    sheets_data = {}

    print(file_path)

    # Filtering Visible Sheets and Ignoring Hidden Sheets
    visible_sheets = [sheet for sheet in wb.sheetnames if wb[sheet].sheet_state == 'visible']

    print(f"Sheets : {visible_sheets}")

    # Loop through all sheets except "MainSheet"
    for sheet_name in visible_sheets:
        if not is_valid_email(sheet_name):
            continue  # Skip "MainSheet"
        sheet = wb[sheet_name]  # Get the sheet
    
        # Extract headers (first row)
        headers = [cell.value for cell in sheet[1]]  # First row as header
    
        # Identify valid columns (Ignore blank headers)
        valid_columns = [i for i, h in enumerate(headers) if h and str(h).strip() != ""]

        source=f'{file_path} : {sheet}'

        # If no valid columns, skip the sheet
        if not valid_columns:
            print(f"Skipping sheet '{sheet_name}' due to no valid headers.")
            continue

        # Prepare filtered headers
        filtered_headers = [str(headers[i]) for i in valid_columns]
        print(f'filter_headers : {filtered_headers} \n')
        filtered_headers=[feature.strip() for feature in filtered_headers]
        filtered_headers.append('source')

        # Prepare data list
        data = []

        # Iterate over rows (starting from the second row)
        for row in sheet.iter_rows(min_row=2, values_only=False):
            row_data = []
            empty_row = True  # Track if the row is empty

            for i in valid_columns:  # Only process valid columns
                cell = row[i]
                cell_value = str(cell.value)
                

                # Append annotation comments to values
                if cell.comment:
                    comment_text = cell.comment.text.strip()
                    if cell_value:
                        cell_value = f"{cell_value} # {comment_text}"
                    else:
                        cell_value = f"({comment_text})"
                        
                    cell.comment = None  # Remove comment after processing

                row_data.append(cell_value)
                

                # If any cell has a value, mark row as non-empty
                if cell_value not in [None, "", " "]:
                    empty_row = False
                


            # Skip completely empty rows
            if not empty_row:
                row_data.append(source)
                data.append(row_data)

            
        

        # If no valid rows exist after filtering, skip the sheet
        if not data:
            print(f"Skipping sheet '{sheet_name}' because it has no valid data.")
            continue


        
        

        df = pd.DataFrame(data, columns=filtered_headers)
        df.dropna(axis=1, how="all", inplace=True)
        print(f"{sheet_name} : {df.shape}")
        print(f"{sheet_name} : {filtered_headers}")
        print("\n ================================================================================== \n")
        # sheets_data[sheet_name] = df  # Store DataFrame in dictionary

        drop_headers=['Column D','Title']
        for feature in filtered_headers:
            if feature in drop_headers:
                 df.drop(columns=[feature],inplace=True)

        # if 'title' in [feature.lower() for feature in filtered_headers]:
        #     df.drop(columns=['Title'],inplace=True)
        df.columns=df.columns.str.replace(' ','')
        df_melted = df.melt(id_vars=['FirstName', 'CompanyName', 'mail', 'Email', 'CTA', 'Domain', 'ID','source'], 
                     var_name="date", 
                     value_name="raw_status")
    
        sheets_data[sheet_name] = df_melted

    wb.close()
    print("Process Complete \n")
    return pd.concat(sheets_data.values(), ignore_index=True)

In [5]:
april_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/April 2024/')
april_2024.shape
april_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/april_2024.csv')
del april_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/April 2024/
File Paths : ['Schneider_Life_Sciences_Spring_Del2_3_09April_2024-13_April_2024.xlsx', 'Schneider_Life_Sciences_Spring_Del2_3_09April_2024-10_April_2024.xlsx', 'Schneider_Life_Sciences_Easter_Del2_3_27Mar_2024-05_April_2024.xlsx', 'SE_LifeSciences_Del1_2_4_Spring_16Apr24-19_April_2024.xlsx', 'Schneider_Life_Sciences_Easter_Del2_3_27Mar_2024-01_April_2024.xlsx', 'Schneider_Life_Sciences_Spring_Del2_3_09April_2024-15_April_2024.xlsx', 'SE_LifeSciences_Del1_2_4_Spring_16Apr24-20_April_2024.xlsx', 'SE_Life_Sciences_Spring_23April_2024-26_April_2024.xlsx', 'SE_LifeSciences_Del1_2_4_Spring_16Apr24-16_April_2024.xlsx', 'Schneider_Life_Sciences_Spring_Del2_3_09April_2024-12_April_2024.xlsx', 'SE_Life_Sciences_Spring_23April_2024-24_April_2024.xlsx', 'Schneider_Life_Sciences_Easter_Del2_3_27Mar_2024-03_April_2024.xlsx', 'SE_Life_Sciences_Spring_Del5_24April_2024-25_Apr

1065887

In [6]:
august_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/August 2024/')
august_2024.shape
august_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/august_2024.csv')
del august_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/August 2024/
File Paths : ['Schneider_Airport_Delivery3_30JULY_2024-03_August_2024.xlsx', 'Schneider_BMS_All_DEL_26th_August_2024-28_August_2024.xlsx', 'Schneider_SemiConductor_All_Deliveries_22Aug_2024-23_August_2024.xlsx', 'Schneider_SemiConductor_micron_texas_02August_2024-07_August_2024.xlsx', 'Schneider_EV_Del _1to4_30thJuly_2024-17_August_2024.xlsx', 'Schneider_EV_Del _1to4_30thJuly_2024-02_August_2024.xlsx', 'Schneider_OEM_Pumps_Del_1to4_30July_2024-03_August_2024.xlsx', 'Schneider_SemiConductor_All_Deliveries_28Aug_2024-29_August_2024.xlsx', 'Schneider_BMS_DEL_1_22th_August_2024-24_August_2024.xlsx', 'Schneider_SemiConductor_micron_texas_02August_2024-06_August_2024.xlsx', 'Schneider_SemiConductor_del5_6_31July_2024-03_August_2024.xlsx', 'SE_Semiconductor_Director+_26Aug24-28_August_2024.xlsx', 'Schneider_SemiConductor_All_Deliveries_6Aug_2024-21_August_2024.xlsx'

582290

In [7]:
december_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/December 2024/')
december_2024.shape
december_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/december_2024.csv')
del december_2024
gc.collect()


Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/December 2024/
File Paths : ['SE_BMS_US_Revised_Del1-3_10th_Dec_2024-21_Nov_2024.xlsx', 'Schneider_Robotics_Del1_2_3_19thNov_2024-03_Nov_2024.xlsx', 'SE_Microgrid_Responder_AC_03rd_Dec_2024-19_Dec_2024.xlsx', 'SE_Microgrid_Responder_AC_03rd_Dec_2024-30_Dec_2024.xlsx', 'SE_BMS_US_Revised_Del1&2_27th_Nov_2024-06_Dec_2024.xlsx', 'Schneider_Robotics_Responder_04Dec_2024-04_Nov_2024.xlsx', 'SE_Mexico_Delivery1-3_24Dec24-26_Nov_2024.xlsx', 'SE_BMS_US_Revised_Del1-3_10th_Dec_2024-13_Nov_2024.xlsx', 'SE_Microgrid_Non_Responder_AC_New_03rd_Dec_2024-24_Dec_2024.xlsx', 'SE_Microgrid_Non_Responder_AC_New_03rd_Dec_2024-11_Dec_2024.xlsx', 'SE_BMS_US_Revised_Del1&2_27th_Nov_2024-04_Dec_2024.xlsx', 'Schneider_Robotics_Responder_04Dec_2024-09_Nov_2024.xlsx', 'SE_Mexico_Delivery1&2_12Dec24-13_Nov_2024.xlsx', 'SE_Microgrid_Responder_AC_03rd_Dec_2024-23_Dec_2024 2.xlsx', 'SE_Microgrid_Non_Re

373511

In [8]:
february_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/February 2024/')
february_2024.shape
february_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/february_2024.csv')
del february_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/February 2024/
File Paths : ['Schneider_F&B_FebGeneric_23Feb_2024-28_Feb_2024.xlsx', 'Schneider_Life_Science_FebGeneric_29Feb_2024-29_Feb_2024.xlsx', 'Schneider_F&B_FebGeneric_23Feb_2024-27_Feb_2024.xlsx', 'Schneider_F&B_FebGeneric_23Feb_2024-24_Feb_2024.xlsx', 'Schneider_F&B_FebGeneric_23Feb_2024-29_Feb_2024.xlsx']
/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/February 2024/Schneider_F&B_FebGeneric_23Feb_2024-28_Feb_2024.xlsx
Sheets : ['Data', 'Data 1', 'Sheet9', 'Using data', 'vlookups', 'URl', 'IDs', 'Power Bi', 'Daily report', 'Mainsheet', 'njohn@insightlab.co', 'white.g@insightlab.co', 'w.gary@insightlab.co']
filter_headers : ['First Name', 'CompanyName', 'mail', 'Email', 'Title', 'CTA', 'Domain', 'ID', '2024-02-24 00:00:00', '2024-02-27 00:00:00', '2024-02-28 00:00:00', '2024-02-29 00:00:00', '2024-03-01 00:00:

578388

In [9]:
february_2025 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/February 2025/')
february_2025.shape
february_2025.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/february_2025.csv')
del february_2025
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/February 2025/
File Paths : ['SE_Mexico_Delivery1-4_24_Jan_2025-03_Feb_2025.xlsx', 'SE_BMS_US_28th_Jan_2025-01_Feb_2025.xlsx', 'SE_BMS_US_3_Feb_2025-11_Feb_2025.xlsx', 'SE_Mexico_spanish_survey_7_Feb_2025-11_Feb_2025.xlsx', 'SE_BMS_US_3_Feb_2025-05_Feb_2025.xlsx', 'SE_Mexico_spanish_survey_7_Feb_2025-07_Feb_2025.xlsx', 'SE_Mexico_Lifesciences_7_Feb_2025-10_Feb_2025.xlsx', 'SE_Mexico_Delivery1-4_24_Jan_2025-04_Feb_2025.xlsx', 'SE_Mexico_Delivery1-4_24_Jan_2025-01_Feb_2025.xlsx', 'SE_Mexico_Lifesciences_12_Feb_2025-12_Feb_2025.xlsx', 'SE_Mexico_spanish_survey_7_Feb_2025-10_Feb_2025.xlsx', 'SE_BMS_US_3_Feb_2025-12_Feb_2025.xlsx', 'SE_Mexico_spanish_survey_7_Feb_2025-08_Feb_2025.xlsx', 'SE_Mexico_Food and beverage_7_Feb_2025-10_Feb_2025.xlsx', 'SE_Mexico_Lifesciences_7_Feb_2025-11_Feb_2025.xlsx', 'SE_BMS_US_3_Feb_2025-08_Feb_2025.xlsx', 'SE_Mexico_Lifesciences_7_Feb_2025-07_F

1911931

In [10]:
january_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/January 2024/')
january_2024.shape
january_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/january_2024.csv')
del january_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/January 2024/
File Paths : ['SE_JanGeneric_2024-05_Jan_2024.xlsx', 'SE_JanGeneric_2024-04_Jan_2024.xlsx', 'SE_JanGeneric_19Jan_2024-20_Jan_2024.xlsx', 'SE_JanGeneric_Del3_05Jan_2024-05_Jan_2024.xlsx', 'SE_JanGeneric_2024-08_Jan_2024.xlsx', 'SE_JanGeneric_10Jan_2024-11_Jan_2024.xlsx', 'SE_JanGeneric_19Jan_2024-25_Jan_2024.xlsx', 'SE_JanGeneric_Del3_05Jan_2024-06_Jan_2024.xlsx', 'SE_JanGeneric_18Jan_2024-18_Jan_2024.xlsx', 'SE_JanGeneric_19Jan_2024-24_Jan_2024.xlsx', 'SE_JanGeneric_2024-03_Jan_2024.xlsx', 'SE_HappyHolidays_Del2_22_DEC_2023-02_Jan_2024.xlsx', 'SE_JanGeneric_2024-06_Jan 2024.xlsx', 'SE_JanGeneric_Del3_05Jan_2024-08_Jan_2024.xlsx', 'SE_JanGeneric_10Jan_2024-10_Jan_2024.xlsx', 'SE_JanGeneric_19Jan_2024-22_Jan_2024.xlsx', 'SE_JanGeneric_10Jan_2024-12_Jan_2024.xlsx', 'SE_JanGeneric_10Jan_2024-17_Jan_2024.xlsx', 'SE_13_DEC_2023_HappyHolidays-02_Jan_2024.xlsx', 'SE

1160952

In [11]:
july_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/July 2024/')
july_2024.shape
july_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/july_2024.csv')
del july_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/July 2024/
File Paths : ['Schneider_OEM_Pumps_Generic_16July_2024-17_July_2024.xlsx', 'Schneider_OEM_Pumps_Generic_16July_2024-24_July_2024.xlsx', 'Schneider_SemiConductor_4th_of_ July_AllDelivery_2July_2024-12_July_2024.xlsx', 'Schneider_OEM_Pumps_Generic_16July_2024-29_July_2024.xlsx', 'Schneider_OEM_Pumps_Generic_16July_2024-22_July_2024.xlsx', 'Schneider_OEM_Pumps_Independence_Day_2July_2024-05_July_2024.xlsx', 'Schneider_OEM_Pumps_Generic_16July_2024-27_July_2024.xlsx', 'Schneider_Airport_AllDelivery_2JULY_2024-27_July_2024.xlsx', 'Schneider_OEM_Pumps_Generic_25June_2024-01_July_2024.xlsx', 'Schneider_OEM_Pumps_SummerSlump_09July_2024-09_July_2024.xlsx', 'Schneider_SemiConductor_Generic_AllDelivery_23July_2024-26_July_2024.xlsx', 'Schneider_OEM_Pumps_Independence_Day_2July_2024-06_July_2024.xlsx', 'Schneider_SemiConductor_4th_of_ July_AllDelivery_2July_2024-15_July_2

629666

In [12]:
march_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/March 2024/')
march_2024.shape
march_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/march_2024.csv')
del march_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/March 2024/
File Paths : ['Schneider_Life_Science_FebGeneric_29Feb_2024 - 4th March 2024.xlsx', 'Schneider_F_B_MarGeneric_Del1_05Mar_2024 - 7th March 2024.xlsx', 'Schneider_F_B_Easter_Del3_27Mar_2024 - 29th March 2024.xlsx', 'Schneider_Life_Science_FebGeneric_29Feb_2024 - 8th March 2024.xlsx', 'Schneider_F_B_Easter_Del1_2_27Mar_2024 - 30th March 2024.xlsx', 'Schneider_F&B_FebGeneric_23Feb_2024 - 1st March 2024.xlsx', 'Schneider_Life_Sciences_Del1_2_14Mar_2024 -16th March 2024.xlsx', 'Schneider_Life_Sciences_Easter_Del2_3_27Mar_2024 - 29th March 2024.xlsx', 'Schneider_F_B_MarGeneric_Del1_2_12Mar_2024 - 12th March 2024.xlsx', 'Schneider_F_B_MarGeneric_Del1_2_12Mar_2024 - 19th March 2024.xlsx', 'Schneider_LifeScience_Del1_2_MarGen_12Mar_2024 - 12th March 2024.xlsx', 'Schneider_F_B_MarGeneric_Del1_2_12Mar_2024 - 26th March 2024.xlsx', 'Schneider_Life_Science_FebGeneric_29Feb_

1631551

In [13]:
may_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/May 2024/')
may_2024.shape
may_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/may_2024.csv')
del may_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/May 2024/
File Paths : ['Schneider_SemiConductor_Summer_Day_28May_2024 - 30th May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 16th May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 18th May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 17th May 2024.xlsx', 'Schneider_EV_Summer_29May_2024 - 30th May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 22nd May 2024.xlsx', 'Schneider_SemiConductor_Summer_Day_28May_2024 - 31st May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 27th May 2024.xlsx', 'Schneider_SemiConductor_Del_2_Summer_Day_31May_2024 - 31st May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 21st May 2024.xlsx', 'Schneider_EV_MemorialDay_16May_2024 - 28th May 2024.xlsx', 'Schneider_SemiConductor_Summer_Day_28May_2024 -29th May 2024..xlsx', 'Schneider_EV_Summer_29May_2024 - 29th May 2024.xlsx', 'Schneider_EV_Summer_29May_2024 - 31st May 2

466530

In [14]:
october_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/October 2024/')
october_2024.shape
october_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/october_2024.csv')
del october_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/October 2024/
File Paths : ['Schneider_Airport_Director+_23rd_October_2024 - 29th Oct 2024.xlsx', 'SE_Microgrids_One_Step_Away_18Oct_2024 - 18th Oct 2024.xlsx', 'Schneider_Airport_Director+_23rd_October_2024 - 24th Oct 2024.xlsx', 'SE_Microgrids_One_Step_Away_18Oct_2024 - 19th Oct 2024.xlsx', 'Schneider_Airport_Director+_23rd_October_2024 - 30th Oct 2024.xlsx']
/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/October 2024/Schneider_Airport_Director+_23rd_October_2024 - 29th Oct 2024.xlsx
Sheets : ['Data', 'URL', 'Daily Report', 'Power BI', 'amy.carter@insightpanel.biz']
filter_headers : ['First Name', 'CompanyName', 'mail', 'Email', 'CTA', 'Domain', 'ID', '23-Oct-2024', '24-Oct-2024', '2024-10-25 00:00:00', '28-Oct-2024', '29-Oct-2024', '2024-10-30 00:00:00', '2024-11-01 00:00:00'] 

amy.carter@insightpanel.biz : (24, 1

130513

In [15]:
september_2024 = get_processed_dataset('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/September 2024/')
september_2024.shape
september_2024.to_csv('/home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/september_2024.csv')
del september_2024
gc.collect()

Folder Path : /home/darshan/Documents/Schneider Electric Data/Schneider Electric Email Data/SE_Leadgen Tracker File/September 2024/
File Paths : ['Schneider_Airport_San_Francisco_Airport_20th_September_2024-30_Sept_2024.xlsx', 'SE_Microgrids_Del1and2_25Sep_2024-30_Sept_2024.xlsx', 'SE_Robotics_6thSep_2024-09_Sept_2024.xlsx', 'SE_Airport_Director+_10_September_2024-12_Sept_2024.xlsx', 'SE_Microgrids_Del1and2_25Sep_2024-27_Sept_2024.xlsx', 'SE_OEM_10_September_2024-10_Sept_2024.xlsx', 'Schneider_Airport_ATL_AIRPORT_20th_September_2024-30_Sept_2024.xlsx', 'Schneider_Airport_Dallas_Airport_20th_September_2024-24_Sept_2024.xlsx', 'SE_Robotics1&2_11Sep_2024-19_Sept_2024.xlsx', 'SE_OEM_11_September_2024-14_Sept_2024.xlsx', 'Schneider_Airport_Delivery4_4th_September_2024-09_Sept_2024.xlsx', 'Schneider_Airport_Delivery4_10th_September_2024-14_Sept_2024.xlsx', 'Schneider_Airport_ATL_AIRPORT_20th_September_2024-27_Sept_2024.xlsx', 'Schneider_Airport_Dallas_Airport_20th_September_2024-20_Sept_2024

889340